In [ ]:
from sqltoolkit.connectors import AzureSQLConnector
from sqltoolkit.client import DatabaseClient
from sqltoolkit.entities import Table, TableColumn
  
# Azure SQL Connection  
server = 'vince-h-sql.database.windows.net'  
database = 'adventureworks'  
azure_connector = AzureSQLConnector(server=server, database=database)  
sql_client = DatabaseClient(azure_connector)
  

In [ ]:
table = Table(name='SalesLT.ProductModel',
                 db_client=sql_client)

table.get_columns()
table.extract_column_values()

In [4]:
table.columns

[TableColumn(name='ProductModelID', type='int', description=None, sample_values=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], distinct_values=None, primary_key=False),
 TableColumn(name='Name', type='nvarchar', description=None, sample_values=['All-Purpose Bike Stand', 'Bike Wash', 'Cable Lock', 'Chain', 'Classic Vest', 'Cycling Cap', 'Fender Set - Mountain', 'Front Brakes', 'Front Derailleur', 'Full-Finger Gloves'], distinct_values=None, primary_key=False),
 TableColumn(name='CatalogDescription', type='xml', description=None, sample_values=[], distinct_values=None, primary_key=False),
 TableColumn(name='rowguid', type='uniqueidentifier', description=None, sample_values=['0481D7E1-4970-4EFA-A560-020F6579918D', 'CA18ECFD-2023-4FA7-A556-0321153BCA34', '219E2F87-26A9-483B-B968-04578E943096', '0D48C51D-7603-4010-9265-0491805BB010', 'F06999A1-3AA7-4E85-B8CB-049EB2C391FA', 'F570E0D1-E978-4FF2-B5B1-08F01AB60219', 'E3CDC5DD-27C3-4891-9D5E-0D46D1B8457F', 'D71BD21C-239E-4C2B-98A3-101962D6B2D3', '45FE0D77-664

In [7]:
table.dict(exclude=['db_client'])

{'name': 'SalesLT.ProductModel',
 'business_readable_name': None,
 'description': None,
 'columns': [{'name': 'ProductModelID',
   'type': 'int',
   'description': None,
   'sample_values': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
   'distinct_values': None,
   'primary_key': False},
  {'name': 'Name',
   'type': 'nvarchar',
   'description': None,
   'sample_values': ['All-Purpose Bike Stand',
    'Bike Wash',
    'Cable Lock',
    'Chain',
    'Classic Vest',
    'Cycling Cap',
    'Fender Set - Mountain',
    'Front Brakes',
    'Front Derailleur',
    'Full-Finger Gloves'],
   'distinct_values': None,
   'primary_key': False},
  {'name': 'CatalogDescription',
   'type': 'xml',
   'description': None,
   'sample_values': [],
   'distinct_values': None,
   'primary_key': False},
  {'name': 'rowguid',
   'type': 'uniqueidentifier',
   'description': None,
   'sample_values': ['0481D7E1-4970-4EFA-A560-020F6579918D',
    'CA18ECFD-2023-4FA7-A556-0321153BCA34',
    '219E2F87-26A9-483B-B968-04578

In [4]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

aoai_endpoint = os.getenv('OPENAI_ENDPOINT')
aoai_key = os.getenv('OPENAI_API_KEY')
aoai_deployment = os.getenv('OPENAI_DEPLOYMENT')

client = AzureOpenAI(azure_endpoint=aoai_endpoint,
    api_key=aoai_key,
    api_version='2024-02-01')

table_summary_prompt = f"""
Summarize below table by the given context.

===Table Schema
{table_schema}

===Sample Rows
{table_sample}

===Response guideline

You shall write a short table description based only on provided information.
Do Not use any column name or data value in the table.
Do not use any adjective to describe the table. For example, the importance of the table, its comprehensiveness or if it is crucial, or who may be using it. For example, you can say that a table contains certain types of data, but you cannot say that the table contains a 'wealth' of data, or that it is 'comprehensive'.
Please also include some potential usecases of the table, e.g. what kind of questions can be answered by the table, what kind of analysis can be done by the table, etc.
"""

messages = [{"role":"system", "content": "You are a data analyst that can help summarize SQL tables."}, 
            {"role": "user", "content": table_summary_prompt}]

response = client.chat.completions.create(
        model=aoai_deployment,
        messages=messages)

response_message = response.choices[0].message
print(response_message)

ChatCompletionMessage(content='This table contains information related to product models. The table includes an identifier for each product model, a name for the product model, an optional catalog description, a unique identifier for each row, and the date when the data was last modified. \n\nPotential use cases of this table include:\n- Tracking different product models and their attributes.\n- Analyzing changes and updates over time by reviewing the modification dates.\n- Ensuring data integrity and uniqueness with unique row identifiers.\n- Associating specific product models with detailed descriptions for catalog purposes.', refusal=None, role='assistant', function_call=None, tool_calls=None)
